In [1]:
3+2

5

In [2]:
import xlwings

In [3]:
xlwings.sheets

Sheets([<Sheet [통합 문서1]Sheet1>])

In [4]:
xlwings.Range("A1")

<Range [통합 문서1]Sheet1!$A$1>

In [5]:
xlwings.Range("A1").expand()

<Range [통합 문서1]Sheet1!$A$1:$B$10>

In [6]:
xlwings.Range("A1").expand().value

[['message', 'browser'],
 ['첫 로그', 'chrome'],
 ['두번째 로그', 'chrome'],
 ['세번째 로그', 'chrome'],
 ['로그#4', 'chrome'],
 ['로그#5', 'chrome'],
 ['로그#6', 'chrome'],
 ['로그#7', 'chrome'],
 ['로그#8', 'chrome'],
 ['로그#9', 'chrome']]

In [7]:
total_rows = xlwings.Range("A1").expand().value
total_rows

[['message', 'browser'],
 ['첫 로그', 'chrome'],
 ['두번째 로그', 'chrome'],
 ['세번째 로그', 'chrome'],
 ['로그#4', 'chrome'],
 ['로그#5', 'chrome'],
 ['로그#6', 'chrome'],
 ['로그#7', 'chrome'],
 ['로그#8', 'chrome'],
 ['로그#9', 'chrome']]

In [8]:
column_names = total_rows[0]
column_names

['message', 'browser']

In [9]:
total_rows[1:]

[['첫 로그', 'chrome'],
 ['두번째 로그', 'chrome'],
 ['세번째 로그', 'chrome'],
 ['로그#4', 'chrome'],
 ['로그#5', 'chrome'],
 ['로그#6', 'chrome'],
 ['로그#7', 'chrome'],
 ['로그#8', 'chrome'],
 ['로그#9', 'chrome']]

In [16]:
rows_data = [
    dict(zip(column_names, row))
    for row in total_rows[1:]
]
rows_data

[{'message': '첫 로그', 'browser': 'chrome'},
 {'message': '두번째 로그', 'browser': 'chrome'},
 {'message': '세번째 로그', 'browser': 'chrome'},
 {'message': '로그#4', 'browser': 'chrome'},
 {'message': '로그#5', 'browser': 'chrome'},
 {'message': '로그#6', 'browser': 'chrome'},
 {'message': '로그#7', 'browser': 'chrome'},
 {'message': '로그#8', 'browser': 'chrome'},
 {'message': '로그#9', 'browser': 'chrome'}]

# 정리

In [17]:
def get_rows_data():
    total_rows = xlwings.Range("A1").expand().value
    column_names = total_rows[0]
    rows_data = [
        dict(zip(column_names, row))
        for row in total_rows[1:]
    ]
    return rows_data

get_rows_data()

[{'message': '첫 로그', 'browser': 'chrome'},
 {'message': '두번째 로그', 'browser': 'chrome'},
 {'message': '세번째 로그', 'browser': 'chrome'},
 {'message': '로그#4', 'browser': 'chrome'},
 {'message': '로그#5', 'browser': 'chrome'},
 {'message': '로그#6', 'browser': 'chrome'},
 {'message': '로그#7', 'browser': 'chrome'},
 {'message': '로그#8', 'browser': 'chrome'},
 {'message': '로그#9', 'browser': 'chrome'}]

# 데이터베이스 접속

In [19]:
DATABASE = {
    "host":"127.0.0.1",
    "user":"root",
    "password":"1234",
    "db":"myproj11_db",
    "charset":"utf8",
}

In [21]:
import pymysql

connection = pymysql.connect(**DATABASE)
cursor = connection.cursor()
cursor.execute("SELECT * FROM log;")
print(cursor.fetchall())
cursor.close()


()


# with 절

In [37]:
import pymysql
from contextlib import contextmanager

@contextmanager
def db_cursor():
    connection = pymysql.connect(**DATABASE)
    cursor = connection.cursor()
    try:
        yield cursor
    finally:
        cursor.close()

In [38]:
with db_cursor() as cursor:
    cursor.execute("SELECT * FROM log;")
    print(cursor.fetchall())

()


In [41]:
column_data = {'message': '첫 로그', 'browser': 'chrome'}

with db_cursor() as cursor:
    for row in get_rows_data():
        sql = "INSERT INTO log(message, browser) VALUES(%(message)s, %(browser)s)"
        cursor.execute(sql, ["첫 로그", "chrome"])

TypeError: format requires a mapping

In [28]:
get_rows_data()

[{'message': '첫 로그', 'browser': 'chrome'},
 {'message': '두번째 로그', 'browser': 'chrome'},
 {'message': '세번째 로그', 'browser': 'chrome'},
 {'message': '로그#4', 'browser': 'chrome'},
 {'message': '로그#5', 'browser': 'chrome'},
 {'message': '로그#6', 'browser': 'chrome'},
 {'message': '로그#7', 'browser': 'chrome'},
 {'message': '로그#8', 'browser': 'chrome'},
 {'message': '로그#9', 'browser': 'chrome'}]

# SQL injection 공격의 예

In [59]:
with db_cursor() as cursor:
    log_id = '4'
    # sql = "SELECT * FROM log WHERE id = " + str(log_id)
    # sql = "SELECT * FROM log WHERE id = 4 or 1"
    sql = "SELECT * FROM log WHERE id = %s"
    cursor.execute(sql, [log_id])
    for row in cursor:
        print(row)

(4, 'ad', 'ad')


# 데이터 매핑의 예

In [61]:
with db_cursor() as cursor:
    cursor.execute("SELECT * FROM log")
    rows_data = cursor.fetchall()
    for row in cursor:
#        row[0] # id
#        row[1] # message
#        row[2] # chrome
        print(row)

rows_data

((4, 'ad', 'ad'), (5, 'num5', 'num5.com'))

## dict 으로의 변환

## namedtuple

In [64]:
from collections import namedtuple
Log = namedtuple('Log', 'id message browser')

In [67]:
log = (1, 'message','chrome')
print(log[0])
print(log[1])
print(log[2])

1
message
chrome


In [68]:
log = Log(1, 'message','chrome')
print(log[0])
print(log[1])
print(log[2])

1
message
chrome


In [69]:
log.id, log.message, log.browser

(1, 'message', 'chrome')

In [72]:
with db_cursor() as cursor:
    cursor.execute("SELECT * FROM log")
    rows_data = [
        Log(*row)
        for row in cursor.fetchall()
    ]
    
rows_data

[Log(id=4, message='ad', browser='ad'),
 Log(id=5, message='num5', browser='num5.com')]